# EDA y liempieza de datos para posterior carga

In [ ]:
import pandas as pd
import numpy as np
import json
import ast

Abrir los archivos Json para posterior carpeta(todo esto lo hice en una carpeta que no es donde tengo el repositorio Clonado)

In [ ]:
#Se abre el primer archivo
rows=[]
with open ('australian_user_reviews.json',encoding='MacRoman') as f:  # MacRoman es el tipo de encoding que acepta 
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_user=pd.DataFrame(rows)

In [ ]:
#Se abre el de items
rows=[]
with open ('australian_users_items.json',encoding='MacRoman') as f:  # MacRoman es el tipo de encoding que acepta 
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_items=pd.DataFrame(rows)

In [ ]:
#archivo de steams
rows = []
with open('output_steam_games.json', encoding='utf-8') as f:
    for line in f:
        rows.append(json.loads(line))

df_games = pd.DataFrame(rows)

Los abrimos para ver su forma 

In [ ]:
df_games

In [ ]:
df_items

In [ ]:
df_user

Vamos a trabajar primero el df_games 

In [ ]:
#Elimino todas las filas que son completamentes nulas
df_games_clean=df_games.dropna(how='all')

In [ ]:
#Hago one-hot-encoding sobre generos 
df_cleaned = df_games.dropna(subset=['genres'])

# Aplicar one-hot encoding a la columna 'genres'
genres_dummies = pd.get_dummies(df_cleaned['genres'].apply(pd.Series).stack()).groupby(level=0).sum()

# Combinar los datos originales con las columnas codificadas
df_encoded = pd.concat([df_cleaned, genres_dummies], axis=1)

Elimino las columnas que no voy a utilizar para las futuras funciones 

In [ ]:
df_encoded=df_encoded.drop(['title','url','tags','reviews_url','genres','publisher','specs','earlyaccess'], axis=1)

Vamos a trabajar con los formatos 

In [ ]:
df_encoded.info()

Trasnformo precios para poder usar price como float para eso saco todo los datos que puede ser strin

In [ ]:
# Reemplazar valores en la columna 'price'
df_encoded['price'] = df_encoded['price'].replace(['Free to Play', 'Free To Play','Free','0 Demo','Play for 0!','Install Now','Play WARMACHINE: Tactics Demo',
                                         '0 Mod','Install Theme','Third-party','Play Now','0 HITMAN™ Holiday Pack','Play the Demo','0 to Try','0 to Use','0 Demo'], '0', regex=True)

#Elimino nulos 
df_encoded['price']=df_encoded['price'].dropna()

#Cambio de formato

df_encoded['price'] = df_encoded['price'].astype(float)

Voy a trabajar los ID como enteros 

In [ ]:
df_encoded['item_id'] = df_encoded['item_id'].astype(int)

Voy a trabajar las columnas voy a pasar todo a minusculas para ser mas facil validar datos

In [ ]:
import re

# Cambiar nombres de columnas a minúsculas
df_encoded.columns = df_encoded.columns.str.lower()

# Función para eliminar caracteres especiales de una cadena
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Aplicar la función para eliminar caracteres especiales en los nombres de columnas
df_encoded.columns = df_encoded.columns.map(remove_special_characters)

# Mostrar las primeras filas del DataFrame con los nuevos nombres de columnas
print(df_encoded.head())

Se cambia nombre de id a item_id y paso a minuscula la columna developer

In [ ]:
df_encoded.rename(columns={'itemid': 'item_id'}, inplace=True)
df_encoded['item_id'] = df_encoded['item_id'].str.rstrip('.0')

In [ ]:
df_encoded['developer'] = df_encoded['developer'].str.lower()

# Eliminar caracteres extraños (solo conservar letras y números)
df_encoded['developer'] = df_encoded['developer'].str.replace('[^a-zA-Z0-9 ]', '')

Voy a trabajar con items empezando con desanidarla columna items 

In [ ]:
df_items_expanded_list = []

#Procesamos cada fila del DataFrame original por separado
for idx, row in df_items.iterrows():
    # Expandimos la lista de 'items' en una fila en un DataFrame separado
    temp = pd.json_normalize(row['items'])
    # Agregamos 'user_id' al DataFrame temporal
    temp['user_id'] = row['user_id']
    # Agregamos el DataFrame temporal a nuestra lista de resultados
    df_items_expanded_list.append(temp)

# Finalmente, concatenamos todos los DataFrames temporales en uno solo
df_items_expanded2 = pd.concat(df_items_expanded_list, ignore_index=True)

In [ ]:
#Veo la data 
df_items_expanded2.head()

Elimino las columnas que no voy a usar

In [ ]:
df_items_expanded2=df_items_expanded2.drop(['item_name','playtime_2weeks'], axis=1)

Por el momento dejo items asi y trabajo con user

In [ ]:
df_user_expanded_list = []

# Luego, procesamos cada fila del DataFrame original por separado
for idx, row in df_user.iterrows():
    # Expandimos la lista de 'items' en una fila en un DataFrame separado
    temp = pd.json_normalize(row['reviews'])
    # Agregamos 'user_id' al DataFrame temporal
    temp['user_id'] = row['user_id']
    # Agregamos el DataFrame temporal a nuestra lista de resultados
    df_user_expanded_list.append(temp)

# Finalmente, concatenamos todos los DataFrames temporales en uno solo
df_user_expanded = pd.concat(df_user_expanded_list, ignore_index=True)

In [ ]:
df_user_expanded.head()

Voy a crear setiment score

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


nltk.download('vader_lexicon')

# Inicializa el analizador de sentimiento
sia = SentimentIntensityAnalyzer()

# Función para asignar etiquetas de sentimiento a valores numéricos
def get_sentiment_label(sentiment_score):
    if sentiment_score >= 0.05:
        return 2  # Positivo
    elif sentiment_score <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral

# Función para analizar sentimiento y asignar etiquetas numéricas
def analyze_sentiment(text):
    if not text:  # Verifica si el texto está vacío o None
        return 1  # Etiqueta neutral
    sentiment_score = sia.polarity_scores(text)['compound']
    sentiment_label = get_sentiment_label(sentiment_score)
    return sentiment_label

# Aplica el análisis de sentimiento a la columna de reseñas y crea una nueva columna
df_user_expanded['sentiment_analysis'] = df_user_expanded['review'].apply(analyze_sentiment)


Cambio el formato de recommend para usar mas adelante en las funciones 

In [ ]:
df_user_expanded['recommend'] = df_user_expanded['recommend'].astype(int)

Transformo las fechas para poder trabajar en las funciones elimino nulos para evitar errores

In [ ]:
# Eliminar "Posted", comas y espacios, y convertir a formato de fecha
df_user_expanded['posted'] = df_user_expanded['posted'].str.replace('Posted', '').str.replace(',', '').str.strip()
df_user_expanded['posted'] = pd.to_datetime(df_user_expanded['posted'], errors='coerce')

# Si solo quieres la fecha en formato de cadena, sin la hora
df_user_expanded['posted'] = df_user_expanded['posted'].dt.strftime('%Y-%m-%d')

In [ ]:
reviews_cleaned = df_user_expanded.dropna(subset=['posted'])

Eliminar las columnas que no voy a utilizar en mis funciones

In [ ]:
df_user_expanded=df_user_expanded.drop(['funny','last_edited','helpful','review',], axis=1)

Pongo todo los usuarios en minuscula para que se haga mas facil trabajar las funciones 

In [ ]:
df_user_expanded['user_id'] = df_user_expanded['user_id'].str.lower().astype(str)
df_items_expanded2['user_id'] = df_items_expanded2['user_id'].str.lower().astype(str)

Por un tema de rendimiento voy a hacer unas transformaciones en items y trabajar con menos dataframe en las funciones 

In [ ]:
#Agrupo los items 

df_agrupado = df_items_expanded2.groupby('user_id')['item_id'].agg(list).reset_index()

#Le agrego item_count y url para tener menos dataframe para trabajar en las funciones
df_user=df_user.drop(['reviews'], axis=1)
df_final = pd.merge(df_agrupado, df_user, on='user_id', how='inner')

Creo un Dataframe para tener el tiempo jugado 

In [ ]:
play_time=df_items_expanded2.drop(['item_name','playtime_2weeks'], axis=1)

Con esto reduci mucho el peso de mis archivos y buscaba tener la informacion mas comprimida y los guardo en formato parquet para que se reduzca el peso y trabajar bien desde git hub

In [ ]:
play_time=pd.read_parquet('play_time.parquet')
df_items_expanded2=pd.read_parquet('item.parquet')
df_user_expanded=pd.read_parquet('reviews.parquet')
df_encoded=pd.read_parquet('steam.parquet')

Esto son los archivos que voy a utilizar